In [1]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%matplotlib inline
le = LabelEncoder()

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
test = pd.read_csv('SF_crime/test.csv', index_col='Id')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
test.Dates = pd.to_datetime(test.Dates)
test_keep = test
crime_in_sf = pd.read_csv('SF_crime/train.csv')
crime_in_sf.Dates = pd.to_datetime(crime_in_sf.Dates)
crime_in_sf = crime_in_sf.rename(columns={'X': 'Longitude', "Y": "Latitude",})
crime_in_sf = crime_in_sf.drop(['Resolution', 'Descript'], axis=1)
crime_train, crime_test = train_test_split(crime_in_sf, test_size=.4)

In [3]:
for column in test.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(test[column])
        test[column] = le.transform(test[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])
        crime_train[column] = le.transform(crime_train[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])        
        crime_test[column] = le.transform(crime_test[column])

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
categories = crime_train.Category
crime_train = crime_train.drop('Category', axis=1)

categories2 = crime_test.Category
crime_test = crime_test.drop('Category', axis=1)

In [5]:
dtrain = xgb.DMatrix(crime_train.as_matrix(),
                     label=categories)
dtest = xgb.DMatrix(crime_test.as_matrix(),
                    label=categories2)

In [6]:
param = {'bst:max_depth':6, 'objective':'multi:softprob', 'num_class':39}
param['nthread'] = 4
param['eval_metric'] = ['merror', 'mlogloss']
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 280

In [7]:
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3)

Will train until train error hasn't decreased in 3 rounds.
Multiple eval metrics have been passed: 'mlogloss' will be used for early stopping.

[0]	eval-merror:0.743745	eval-mlogloss:3.103544	train-merror:0.742892	train-mlogloss:3.101094
[1]	eval-merror:0.737936	eval-mlogloss:2.924872	train-merror:0.736882	train-mlogloss:2.920309
[2]	eval-merror:0.736012	eval-mlogloss:2.810064	train-merror:0.734652	train-mlogloss:2.803595
[3]	eval-merror:0.733885	eval-mlogloss:2.727376	train-merror:0.732705	train-mlogloss:2.719426
[4]	eval-merror:0.732803	eval-mlogloss:2.665503	train-merror:0.731363	train-mlogloss:2.655529
[5]	eval-merror:0.732211	eval-mlogloss:2.617824	train-merror:0.730233	train-mlogloss:2.606332
[6]	eval-merror:0.731840	eval-mlogloss:2.581441	train-merror:0.729825	train-mlogloss:2.568170
[7]	eval-merror:0.731012	eval-mlogloss:2.551688	train-merror:0.728999	train-mlogloss:2.536865
[8]	eval-merror:0.730576	eval-mlogloss:2.528611	train-merror:0.728398	train-mlogloss:2.512051
[9]	eval-m

low point: [279]	eval-merror:0.709555	eval-mlogloss:2.365904	train-merror:0.651488	train-mlogloss:2.074684

In [8]:
predictions = bst.predict(xgb.DMatrix(test.as_matrix()), output_margin=False)

In [9]:
predictions = pd.DataFrame(predictions)

In [10]:
le.fit(crime_in_sf.Category)
predictions.columns = le.inverse_transform(predictions.columns)

In [11]:
pred_test = predictions

In [12]:
predictions['Id'] = predictions.index

In [13]:
def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame

predictions = order(predictions,['Id'])

In [14]:
predictions.to_csv('/Users/MatthewBarnette/final_project_predictions//predictions_XGB_280.csv', index=False)